# Architektur Neuronales Netz

In [1]:
# aNN Architektur

# Importe / Bibliotheken
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader, random_split
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path


#### Default Datentyp

In [2]:
torch.set_default_dtype(torch.float64)

#### Erzeugnung des Moduls

In [3]:
class NeuralNetwork(nn.Module):
    
    #Initalisierung der Netzwerk layers
    def __init__(self, input_size, hidden1_size, hidden2_size, output_size):
    
        super().__init__() #Referenz zur Base Class (nn.Module)
        #Kaskade der Layer
        self.linear_afunc_stack = nn.Sequential(
            nn.Linear(input_size, hidden1_size), #Lineare Transformation mit gespeicherten weights und biases
            nn.Tanh(), #Nicht lineare Aktivierungsfunktion um komplexe nichtlineare Zusammenhänge abzubilden
            nn.Linear(hidden1_size, hidden2_size),
            nn.Tanh(),
            nn.Linear(hidden2_size, output_size),
        )

    #Implementierung der Operationen auf Input Daten
    def forward(self, x):
        #x = self.flatten(x)
        out = self.linear_afunc_stack(x)
        return out

#### Ausgabe Modul

In [4]:
model = NeuralNetwork(5, 20, 20, 1)
print(model)

NeuralNetwork(
  (linear_afunc_stack): Sequential(
    (0): Linear(in_features=5, out_features=20, bias=True)
    (1): Tanh()
    (2): Linear(in_features=20, out_features=20, bias=True)
    (3): Tanh()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)


#### DataLoader

In [5]:
batch_size = 32 #Zahl der Datenpaare die vor einem erneuten Update der Parameter ins Netzt gegeben werden
eq_data_file = Path.cwd() / 'data' / 'eq_dataset.npz' #Import der GGW Daten

res = np.load(eq_data_file)

# Bei Speicherung wurden Daten als T, p, x_0 und xi gespeichert
# Inputs T, p, x_0[H2,N2,NH3]
# Outputs xi
# Umwandlen der np.arrays in torch.tensors zur besseren Arbeit mit PyTorch
T = torch.tensor(res['T'])
p = torch.tensor(res['p'])
x_0 = torch.tensor(res['x_0'])
xi = torch.tensor(res['xi'])

# T = torch.tensor(res['T']).float()
# p = torch.tensor(res['p']).float()
# x_0 = torch.tensor(res['x_0']).float()
# xi = torch.tensor(res['xi']).float()

# print(T.dtype)
# print(xi.dtype)

x_input = torch.stack((T,p,x_0[:,0],x_0[:,1],x_0[:,2]),1)
y_output = xi
#print(x_input.size())
# print(xi.size())

# Tensoren zu einem großen Set gruppieren
dataset = TensorDataset(x_input, y_output)

# for x,y in dataset:
#     print(x)
#     print(y)
    
# Split in Trainings und Test Set
train_dataset, test_dataset = random_split(dataset, 
                                           [int(0.8*len(dataset)), int(0.2*len(dataset))], # splitting 80/20
                                           generator = torch.Generator().manual_seed(42)) # Festlegung seed zur Reproduktivität

# Erzeugen der DataLoader zur Arbeit mit Daten
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True) # shuffle batches zur Reduzierung von overfitting
test_dataloader = DataLoader(test_dataset, batch_size = batch_size, shuffle=False)


#### Generierung Netzwerk, Festlegung von loss Funktion und Optimizer

In [6]:
# Erzeugung aNN
net = NeuralNetwork(5, 20, 20, 1)

# Loss Funktion; gibt Fehler an
#loss_fn = nn.MSELoss()

#Definition custom loss Funktion, MRE
def MRELoss(outputs, targets):
    
    loss = torch.mean(abs((outputs - targets) / targets))
    
    return loss
    

#Optimizer
learning_rate = 0.001
optimizer = torch.optim.Adam(net.parameters(), lr = learning_rate)


#### Training

In [48]:
num_epochs = 500 #Iterationen über Datenset

train_loss = []
test_loss = []
train_accuracy = []
test_accuracy = []

#Optimierungsloop
for epoch in range(num_epochs):
    train_correct = 0
    train_total = 0
        
    for batch, (X,y) in enumerate(train_dataloader):
        
#         print(X.shape)
#         print(X.dtype)
        
        net.train() #Trainingmodus
        
        # forward
        pred = net(X)  # Do the forward pass
        #loss = loss_fn(pred, y) # Calculate the loss
        loss = MRELoss(pred, y)
        
        # backward
        optimizer.zero_grad() # Clear off the gradients from any past operation
        loss.backward()       # Calculate the gradients with help of back propagation, updating weights and biases
        
        # adam step gradient descent
        optimizer.step()      # Ask the optimizer to adjust the parameters based on the gradients
        
#         # Record the correct predictions for training data
#         _, predictions = torch.max(pred.data, 1)
#         train_correct += (predictions == y.data).sum()                
#         train_total += predictions.size(0)    

        print ('Epoch %d/%d, Iteration %d/%d, Loss: %.4f' 
               %(epoch+1, num_epochs, batch+1, len(train_dataset)//batch_size, loss.item()))
        
    
    net.eval() # Put the network into evaluation mode
    
    # Book keeping
    # Record the loss
    train_loss.append(loss.item())
    
    # What was our train accuracy?
    train_acc = check_accuracy(train_dataloader, net)
    train_accuracy.append(train_acc)
    
    # How did we do on the test set (the unseen set)
    # Record the correct predictions for test data
    test_acc, test_lost = check_accuracy(test_dataloader, net)
    test_accuracy.append(test_acc)
    test_loss.append(test_lost)   

#     loss = criterion(outputs, Variable(test_classes))
#     test_loss.append(loss.data[0])

Epoch 1/500, Iteration 1/25, Loss: 1.1662
Epoch 1/500, Iteration 2/25, Loss: 0.9727
Epoch 1/500, Iteration 3/25, Loss: 0.9249
Epoch 1/500, Iteration 4/25, Loss: 1.0238
Epoch 1/500, Iteration 5/25, Loss: 0.9796
Epoch 1/500, Iteration 6/25, Loss: 0.9909
Epoch 1/500, Iteration 7/25, Loss: 0.9986
Epoch 1/500, Iteration 8/25, Loss: 0.9996
Epoch 1/500, Iteration 9/25, Loss: 0.9998
Epoch 1/500, Iteration 10/25, Loss: 1.0057
Epoch 1/500, Iteration 11/25, Loss: 1.0004
Epoch 1/500, Iteration 12/25, Loss: 0.9819
Epoch 1/500, Iteration 13/25, Loss: 1.0210
Epoch 1/500, Iteration 14/25, Loss: 1.0047
Epoch 1/500, Iteration 15/25, Loss: 1.0926
Epoch 1/500, Iteration 16/25, Loss: 1.1170
Epoch 1/500, Iteration 17/25, Loss: 1.0042
Epoch 1/500, Iteration 18/25, Loss: 1.0139
Epoch 1/500, Iteration 19/25, Loss: 0.9796
Epoch 1/500, Iteration 20/25, Loss: 1.0004
Epoch 1/500, Iteration 21/25, Loss: 0.9863
Epoch 1/500, Iteration 22/25, Loss: 1.2738
Epoch 1/500, Iteration 23/25, Loss: 0.9828
Epoch 1/500, Iterati

Epoch 8/500, Iteration 16/25, Loss: 1.0794
Epoch 8/500, Iteration 17/25, Loss: 1.2268
Epoch 8/500, Iteration 18/25, Loss: 1.0402
Epoch 8/500, Iteration 19/25, Loss: 1.2389
Epoch 8/500, Iteration 20/25, Loss: 1.2581
Epoch 8/500, Iteration 21/25, Loss: 1.2543
Epoch 8/500, Iteration 22/25, Loss: 1.7699
Epoch 8/500, Iteration 23/25, Loss: 2.3853
Epoch 8/500, Iteration 24/25, Loss: 1.1363
Epoch 8/500, Iteration 25/25, Loss: 1.1456
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.038650 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.057580 

Epoch 9/500, Iteration 1/25, Loss: 1.0233
Epoch 9/500, Iteration 2/25, Loss: 1.1135
Epoch 9/500, Iteration 3/25, Loss: 0.9827
Epoch 9/500, Iteration 4/25, Loss: 0.9965
Epoch 9/500, Iteration 5/25, Loss: 0.9908
Epoch 9/500, Iteration 6/25, Loss: 1.0099
Epoch 9/500, Iteration 7/25, Loss: 1.0204
Epoch 9/500, Iteration 8/25, Loss: 1.0340
Epoch 9/500, Iteration 9/25, Loss: 0.9609
Epoch 9/500, Iteration 10/25, Loss: 1.0362
Epoch 9/500, Iteration 11/25, Loss: 0.8794
Ep

Epoch 16/500, Iteration 4/25, Loss: 1.0300
Epoch 16/500, Iteration 5/25, Loss: 0.9947
Epoch 16/500, Iteration 6/25, Loss: 1.0737
Epoch 16/500, Iteration 7/25, Loss: 1.0032
Epoch 16/500, Iteration 8/25, Loss: 1.0728
Epoch 16/500, Iteration 9/25, Loss: 1.1329
Epoch 16/500, Iteration 10/25, Loss: 1.0034
Epoch 16/500, Iteration 11/25, Loss: 1.7218
Epoch 16/500, Iteration 12/25, Loss: 1.0270
Epoch 16/500, Iteration 13/25, Loss: 1.0021
Epoch 16/500, Iteration 14/25, Loss: 1.0687
Epoch 16/500, Iteration 15/25, Loss: 1.0135
Epoch 16/500, Iteration 16/25, Loss: 0.9399
Epoch 16/500, Iteration 17/25, Loss: 0.9858
Epoch 16/500, Iteration 18/25, Loss: 1.0010
Epoch 16/500, Iteration 19/25, Loss: 0.9434
Epoch 16/500, Iteration 20/25, Loss: 0.9934
Epoch 16/500, Iteration 21/25, Loss: 1.0868
Epoch 16/500, Iteration 22/25, Loss: 0.9430
Epoch 16/500, Iteration 23/25, Loss: 1.0297
Epoch 16/500, Iteration 24/25, Loss: 1.0479
Epoch 16/500, Iteration 25/25, Loss: 1.2577
Train Error: 
 Accuracy: 0.62%, Avg lo

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.065826 

Epoch 25/500, Iteration 1/25, Loss: 0.9771
Epoch 25/500, Iteration 2/25, Loss: 1.1093
Epoch 25/500, Iteration 3/25, Loss: 1.0263
Epoch 25/500, Iteration 4/25, Loss: 1.1152
Epoch 25/500, Iteration 5/25, Loss: 2.4370
Epoch 25/500, Iteration 6/25, Loss: 0.9981
Epoch 25/500, Iteration 7/25, Loss: 0.9862
Epoch 25/500, Iteration 8/25, Loss: 0.9998
Epoch 25/500, Iteration 9/25, Loss: 1.1239
Epoch 25/500, Iteration 10/25, Loss: 1.1200
Epoch 25/500, Iteration 11/25, Loss: 1.0377
Epoch 25/500, Iteration 12/25, Loss: 1.0123
Epoch 25/500, Iteration 13/25, Loss: 1.1027
Epoch 25/500, Iteration 14/25, Loss: 1.0146
Epoch 25/500, Iteration 15/25, Loss: 1.1862
Epoch 25/500, Iteration 16/25, Loss: 1.0122
Epoch 25/500, Iteration 17/25, Loss: 1.1041
Epoch 25/500, Iteration 18/25, Loss: 1.0007
Epoch 25/500, Iteration 19/25, Loss: 0.9287
Epoch 25/500, Iteration 20/25, Loss: 0.9435
Epoch 25/500, Iteration 21/25, Loss: 1.0080
Epoch 25/500, Iteration 22/25, Los

Epoch 34/500, Iteration 13/25, Loss: 1.1106
Epoch 34/500, Iteration 14/25, Loss: 0.9721
Epoch 34/500, Iteration 15/25, Loss: 1.0444
Epoch 34/500, Iteration 16/25, Loss: 1.0002
Epoch 34/500, Iteration 17/25, Loss: 0.9710
Epoch 34/500, Iteration 18/25, Loss: 0.9807
Epoch 34/500, Iteration 19/25, Loss: 1.8334
Epoch 34/500, Iteration 20/25, Loss: 0.9996
Epoch 34/500, Iteration 21/25, Loss: 1.0061
Epoch 34/500, Iteration 22/25, Loss: 0.9891
Epoch 34/500, Iteration 23/25, Loss: 1.5872
Epoch 34/500, Iteration 24/25, Loss: 1.1029
Epoch 34/500, Iteration 25/25, Loss: 1.0065
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.031715 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.037387 

Epoch 35/500, Iteration 1/25, Loss: 1.0123
Epoch 35/500, Iteration 2/25, Loss: 1.0019
Epoch 35/500, Iteration 3/25, Loss: 0.9776
Epoch 35/500, Iteration 4/25, Loss: 0.9279
Epoch 35/500, Iteration 5/25, Loss: 0.9437
Epoch 35/500, Iteration 6/25, Loss: 0.9570
Epoch 35/500, Iteration 7/25, Loss: 0.9024
Epoch 35/500, Iteration

...

Epoch 248/500, Iteration 21/25, Loss: 1.0030
Epoch 248/500, Iteration 22/25, Loss: 0.9885
Epoch 248/500, Iteration 23/25, Loss: 1.0263
Epoch 248/500, Iteration 24/25, Loss: 1.0306
Epoch 248/500, Iteration 25/25, Loss: 1.6172
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.035126 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.048152 

Epoch 249/500, Iteration 1/25, Loss: 1.1217
Epoch 249/500, Iteration 2/25, Loss: 1.2674
Epoch 249/500, Iteration 3/25, Loss: 1.0006
Epoch 249/500, Iteration 4/25, Loss: 0.9814
Epoch 249/500, Iteration 5/25, Loss: 1.1610
Epoch 249/500, Iteration 6/25, Loss: 2.2060
Epoch 249/500, Iteration 7/25, Loss: 1.0402
Epoch 249/500, Iteration 8/25, Loss: 1.0007
Epoch 249/500, Iteration 9/25, Loss: 1.0171
Epoch 249/500, Iteration 10/25, Loss: 1.0263
Epoch 249/500, Iteration 11/25, Loss: 1.0201
Epoch 249/500, Iteration 12/25, Loss: 0.9884
Epoch 249/500, Iteration 13/25, Loss: 0.9955
Epoch 249/500, Iteration 14/25, Loss: 1.0012
Epoch 249/500, Iteration 15/25, Loss: 1.0041
Epoch

Train Error: 
 Accuracy: 0.62%, Avg loss: 0.031461 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.036946 

Epoch 256/500, Iteration 1/25, Loss: 1.0156
Epoch 256/500, Iteration 2/25, Loss: 0.9822
Epoch 256/500, Iteration 3/25, Loss: 1.3408
Epoch 256/500, Iteration 4/25, Loss: 1.0344
Epoch 256/500, Iteration 5/25, Loss: 1.0254
Epoch 256/500, Iteration 6/25, Loss: 1.0005
Epoch 256/500, Iteration 7/25, Loss: 0.9892
Epoch 256/500, Iteration 8/25, Loss: 1.0592
Epoch 256/500, Iteration 9/25, Loss: 1.0304
Epoch 256/500, Iteration 10/25, Loss: 1.0507
Epoch 256/500, Iteration 11/25, Loss: 1.0531
Epoch 256/500, Iteration 12/25, Loss: 1.0090
Epoch 256/500, Iteration 13/25, Loss: 1.0681
Epoch 256/500, Iteration 14/25, Loss: 0.9943
Epoch 256/500, Iteration 15/25, Loss: 0.9960
Epoch 256/500, Iteration 16/25, Loss: 0.9941
Epoch 256/500, Iteration 17/25, Loss: 0.9978
Epoch 256/500, Iteration 18/25, Loss: 0.9464
Epoch 256/500, Iteration 19/25, Loss: 1.0058
Epoch 256/500, Iteration 20/25, Loss: 0.9790
Epoch

Epoch 263/500, Iteration 25/25, Loss: 1.0343
Train Error: 
 Accuracy: 0.62%, Avg loss: 0.040382 

Test Error: 
 Accuracy: 1.5%, Avg loss: 0.068932 

Epoch 264/500, Iteration 1/25, Loss: 1.0363
Epoch 264/500, Iteration 2/25, Loss: 1.1720
Epoch 264/500, Iteration 3/25, Loss: 2.0472
Epoch 264/500, Iteration 4/25, Loss: 1.2920
Epoch 264/500, Iteration 5/25, Loss: 1.0096
Epoch 264/500, Iteration 6/25, Loss: 1.0670
Epoch 264/500, Iteration 7/25, Loss: 1.2348
Epoch 264/500, Iteration 8/25, Loss: 1.1140
Epoch 264/500, Iteration 9/25, Loss: 1.1267
Epoch 264/500, Iteration 10/25, Loss: 1.1750
Epoch 264/500, Iteration 11/25, Loss: 0.9439
Epoch 264/500, Iteration 12/25, Loss: 1.2022
Epoch 264/500, Iteration 13/25, Loss: 1.2196
Epoch 264/500, Iteration 14/25, Loss: 1.0279
Epoch 264/500, Iteration 15/25, Loss: 1.0796
Epoch 264/500, Iteration 16/25, Loss: 1.0428
Epoch 264/500, Iteration 17/25, Loss: 1.0115
Epoch 264/500, Iteration 18/25, Loss: 0.9694
Epoch 264/500, Iteration 19/25, Loss: 1.3812
Epoch

Epoch 271/500, Iteration 2/25, Loss: 1.1949
Epoch 271/500, Iteration 3/25, Loss: 1.1257
Epoch 271/500, Iteration 4/25, Loss: 3.2187
Epoch 271/500, Iteration 5/25, Loss: 1.2521
Epoch 271/500, Iteration 6/25, Loss: 1.6870
Epoch 271/500, Iteration 7/25, Loss: 1.0468
Epoch 271/500, Iteration 8/25, Loss: 1.0554
Epoch 271/500, Iteration 9/25, Loss: 1.0414
Epoch 271/500, Iteration 10/25, Loss: 0.9257
Epoch 271/500, Iteration 11/25, Loss: 1.0142
Epoch 271/500, Iteration 12/25, Loss: 1.8237
Epoch 271/500, Iteration 13/25, Loss: 1.0132
Epoch 271/500, Iteration 14/25, Loss: 1.3057
Epoch 271/500, Iteration 15/25, Loss: 1.1094
Epoch 271/500, Iteration 16/25, Loss: 1.4138
Epoch 271/500, Iteration 17/25, Loss: 1.3157
Epoch 271/500, Iteration 18/25, Loss: 1.1694
Epoch 271/500, Iteration 19/25, Loss: 1.0963
Epoch 271/500, Iteration 20/25, Loss: 1.2431
Epoch 271/500, Iteration 21/25, Loss: 1.0894
Epoch 271/500, Iteration 22/25, Loss: 1.1679
Epoch 271/500, Iteration 23/25, Loss: 0.9795
Epoch 271/500, Ite

KeyboardInterrupt: 

#### Funktion zur Bestimmung der Genauigkeit

In [47]:
def check_accuracy(loader, net):
    
    loss = 0
    train_correct = 0
    train_total = 0
    net.eval() # Put network in evaluation mode
    
    if loader == train_dataloader:
        dataset = "Train"
    else:
        dataset = "Test"
    
    with torch.no_grad():
        for X, y in loader:
            pred = net(X)
            
            loss += MRELoss(pred, y)

            # Record the correct predictions for training data
            _, predictions = torch.max(pred.data, 1)
            #train_correct += (predictions == y.data).sum()
            train_correct += (abs(predictions - y.data) <= 0.001).sum()
            train_total += predictions.size(0)
            #print(predictions.size(0))
            #print(train_total)
            
        # Genauigkeit berechnen
        acc = float(train_correct) / float(train_total) * 100
        acc = round(acc, 2)
        
        loss /= train_total
        print(f"{dataset} Error: \n Accuracy: {acc}%, Avg loss: {loss:>8f} \n")

    net.train()
    
    return acc, loss